In [1]:
%run stdPackages.ipynb
from lpEnergyModels.mBasicInt import * # Load everything from the mBasicInt module
import pickle

*Note: "`self`" is a placeholder that refers to the class MBasicInt or a specific instance of the class.*

# The MBasicInt class

The `MBasicInt` class builds primarily on three other classes: (1) The model structure class `symMaps.ModelShell`, (2) the linear programming class `symMaps.LPSys`, and (3) the database class `pyDbs.SimpleDB`. The relevant packages are documented here: 
* [GitHub: ChampionApe/symMaps](https://github.com/ChampionApe/symMaps)
* [GitHub: ChampionApe/pyDbs](https://github.com/ChampionApe/pyDbs).

The class `MBasicInt(ModelShell`) builds directly on the `symMaps.ModelShell` class. To each model instance, we set an attribute `self.sys` as the instance of the linear programming class `symMaps.LPSys` and a database instance `symMaps.SimpleDB` as the attribute `self.sys.db` (which can be accessed by calling `self.db` as well). 

The relevant Github pages include notebooks that more carefully describe the different classes. Here, we go through three steps: Data, model formulation, and solution.

## 1. Data

The model is an extension of the `MBasic` model, but with enough adjustments that it specified as a new class instead of adding `MBasic` as a parent class. To keep the documentation somewhat succinct, we assume that you have already been through the docs on `MBasic` (parentheses indicate symbols used in plain math below):

**Sets:** The model is defined over the following main indices:

*From MBasic:*
* ```idxGen``` ($i$): Set of electricity generators.
* ```idxF``` ($f$): Set of fuel types that generators may use.
* ```idxEm``` ($em$): Set of emission types included in the model.
* ```idxCons``` ($ic$): Set of consumers in the power market.

*New ones:*
* ```idxHr``` ($h$): Set of short run states (hours).
* ```idxHVTGen``` ($vi$): Set of exogenous hourly variation paths, for generators.
* ```idxHVTCons```($vic$): Set of exogenous hourly variation paths, for consumers.


**Variables:**

*From MBasic:*
The model requires the following data to run:
* ```pFuel(idxF)``` ($p_f$): Fuel price. Default unit: €/GJ. 
* ```uEm(idxF, idxEm)``` ($\phi_{f,em}$): Emission intensity. Default unit: Ton emission/GJ fuel input. 
* ```VOM(idxGen)``` ($c_i^{oth}$): variable operating and maintenance costs. Default unit: €/GJ output.
* ```uFuel(idxF, idxGen)``` ($\mu_{f,i}$): Fuelmix. Default unit: GJ input/GJ output.
* ```taxEm(idxEm)``` ($\tau_{em}$): Tax on emissions. Default unit: €/ton emission output.
* ```genCap(idxGen)``` ($q_i$): Generating capacity. Default unit: GJ/hour.
* ```mwp(idxCons)``` ($u_{ic}$): Marginal willingness to pay for consumers. Default unit: €/GJ. 
* ```load(idxCons)``` ($L_{ic}$): Average hourly unconstrained demand (annual mean).

*New ones:*
* ```uHrCap(idxHr, idxHVTGen)``` ($\gamma_{h, vi}$): Parameter measuring how generating capacity varies over the year for variation type ```idxHVTGen```. Default unit: Hourly-to-maximum GJ/hour ratio. Generally, we expect ```uHrCap```$\in[0,1]$ as it measures the scale of generating capacity in hour ```idxH``` relatively to installed effect (```genCap```).
* ```uHrLoad(idxHr, idxHVTCons)``` ($\gamma_{h, vic}$): Similar to ```uHrCap``` except this measures hourly demand relative to the yearly mean. 

Beyond this, ideally we would also add the following variables on the cost structure for generators (these are not required for solving the model, but for assessing cost-effectiveness afterwards):
* ```FOM(idxGen)``` ($FOM_i$): Fixed operating and maintenance costs. Default unit: (1000 €/(GJ/hour generating capacity)) / year.
* ```INVC(idxGen)``` ($INVC_i$): Annualized investment costs. Default unit: 1000 €/ (GJ/hour generating capacity) / year (annualized).

**Mappings:** In addition to new sets and parameters, we require mappings from generators and consumers to the relevant hourly patterns. These are defined as ```pd.MultiIndex``` instances over relevant sets:

* ```idxGen2HVTGen(idxGen, idxHVTGen)```: Identifies what hourly variation pattern the specific generator follows. 
* ```idxCons2HVTCons(idxCons, idxHVTCons)```: Identifies hourly variation pattern specific demand component follows.

The excel file `data/EX_MBasic_CA.xlsx` in the data repo contains an example of the relevant data outlined above (except for values for the endogenous variables that we are going to solve for). The file `data/EX_MBasicInt_CA.pkl` in the data repo contains a pickled example of the excel data after reading it using the `pyDbs.ExcelSymbolLoadear` class.

## 2. Model formulation

### A. Formal setup

**Formal optimization problem:**

The model solves for vectors of electricity generation ($E_{h,i}$ = `generation`) and demand levels ($D_{h,ic}$ = `demand`) by solving:
$$\begin{align}
    &\max\mbox{ }\sum_h\left(\sum_{ic}u_{ic} D_{h, ic} - \sum_{i} c_i E_{h,i} \right) \tag{1A} \\
    &\text{subject to:} \\
    &\sum_{ic} D_{h,ic} = \sum_i E_{h,i}, \qquad \forall h, \tag{1B} \\
    & E_{h,i}\in[0,q_{h,i}], \quad \text{and} \quad D_{h,ic} \in[0, L_{h,ic}], \tag{1C}
\end{align}$$
and where the marginal costs of generation, $E_i$, are computed from:
$$\begin{align}
    c_i = c_i^{oth} + \sum_f \mu_{f,i} (p_f + \sum_{em} \phi_{f,em}\cdot\tau_{em}) \tag{2}
\end{align}$$

We note that the hourly generating capacities $q_{h,i}$ are given by $q_i * \gamma_{h, vi}$ (installed capacity times hourly variation), where $vi$ is matched to the individual generator via the mapping `idxGen2HVTGen`. Similarly, hourly demand/load restriction $L_{h,ic}$ is given by $L_{ic} * \gamma_{h,vic}$ (average hourly demand times hourly variation), where $vic$ is matched to the consumer via the mapping `idxCons2HVTCons`.

**Augmented form:**

The actual structure of the problem is handled by the `symMaps.LPSys`class, which is designed for Linear Programming (LP) problems. This means that we have to formulate the math in the following canonical form (we'll refer to this as an "augmented form"):
$$\begin{align} 
    &\min_{x} \ c^T\cdot x \tag{3A}\\ 
    &A_{ub}\times x \leq b_{ub} \tag{3B}\\ 
    &A_{eq}\times x  = b_{eq} \tag{3C}\\ 
    &l\leq x\leq u, \tag{3D}
\end{align}$$
where: 
* $x$ is the vector of choice variables of length ($N$).
* $c, l, u$ are coefficient vectors of the same length ($N$).
* $b_{eq}, b_{ub}$ are coefficient vectors of lengths $N_{eq}, N_{ub}$, 
* and $A_{eq}, A_{ub}$ are coefficient matrices of sizes $(N_{eq}\times N)$ and $(N_{ub} \times N)$ respectively.

We generally refer to (3B)-(3C) as variational constraints, and (3D) as domain constraints.

In [2]:
testData = os.path.join(_d['data'],'EX_MBasicInt_CA.xlsx')
data = pyDbs.ExcelSymbolLoader(testData)() # read data in a dict 
m = MBasicInt() # initialize model
[m.db.__setitem__(k, v) for k,v in data.items() if k!='__meta__']; # add data to model database

Save pickle instance:

In [3]:
with open (os.path.join(_d['data'], 'EX_MBasicInt_CA.pkl'), "wb") as file:
    pickle.dump(data, file)

### B. Add model structure -  `self.compile`

The method `self.compile` adds the model in equations (1)-(2) to the linear programming system `self.sys` consistent with the structure in equations (3). This is done in three steps:
1. Specify fundamentals: Provide variables and variational constraints in (1) and what sets/domains they are defined over.
2. An auxiliary calculations step: Provides a way of computing variables before the optimization. 
3. Add/specify parameters $c^T, A_{ub}, A_{eq}, b_{ub}, b_{eq}, l, u$ from equations (3).

####  *i)* Fundamentals - `self.compileMaps()`

The `self.compileMaps` starts by declaring domains of variables (in method `self.initArgs_v`), equality constraints (in method `self.initArgs_eq`), and inequality constraints (in method `self.initArgs_ub`). In this model, we have:
* Two variables: `generation` ($E_{h,i}$) defined over the indices `idxHr`  and `idxGen` ($h,i$) and `demand` defined over `idxHr` and `idxCons` ($h, ic$).
* One constraint: `equilibrium` defined over the index `idxHr` ($h$).

With this structure in place, the `self.compileMaps` method then applies `self.sys.compileMaps` which establishes the global indices for variables and constraints: 

In [4]:
m.compileMaps()
m.sys.eq # look at the mapping from constraint names to their relevant domains

{'equilibrium': Index(['h01', 'h02', 'h03', 'h04', 'h05', 'h06', 'h07', 'h08', 'h09', 'h10',
        'h11', 'h12', 'h13', 'h14', 'h15', 'h16', 'h17', 'h18', 'h19', 'h20',
        'h21', 'h22', 'h23', 'h24'],
       dtype='object', name='idxHr')}

####  *ii)* Auxiliary calculations - `self.updateAux`

This step computes auxiliary parameters used in the model. In this case, we compute marginal costs (`mcHr`), hourly generating capacity (`genCapHr`), and hourly load (`loadHr`) and add them to the model database in `self.db`:

In [5]:
m.updateAux()
m.db('mcHr').unstack('idxGen').head() # inspect the marginal costs that we have added to the database

idxGen,biomass,coal,hydro,natgas,nuclear,solar,wind_offshore,wind_onshore
idxHr,,,,,,,,
h01,14.0,18.712,1.0,16.248,2.105,1.0,1.2,1.0
h02,14.0,18.712,1.0,16.248,2.105,1.0,1.2,1.0
h03,14.0,18.712,1.0,16.248,2.105,1.0,1.2,1.0
h04,14.0,18.712,1.0,16.248,2.105,1.0,1.2,1.0
h05,14.0,18.712,1.0,16.248,2.105,1.0,1.2,1.0


We implement this as a standard method in the class because we expect many scenarios likely involve recomputing this object. 

####  *iii)* Specify parameters for `LPSys` - `self.compileParams`

We specify the parameters for the `LPSys` method going through (1) adding coefficients on variables (`c`, `u`, `l`), (2) equality constraints (`A_eq`, `b_eq`), and (3) inequality constraints (`A_ub`, `b_ub`).

**Coefficients on variables:**

Coefficients on `generation` and `demand` are added in a similar way as in the `MBasic` model (specifying `self.initArgsV_{k}` methods with k = `generation`, `demand`); the only difference now is that both variables are defined over hours as well. This is captured either by referencing parameters in the `self.db` that are now defined over hours (e.g. `mcHr[idxHr, idxGen]` instead of `mc[idxGen]`) or by broadcasting a parameter to the relevant domains.

For the `demand` variable, for instance, we add the following:

```python
def initArgsV_demand(self):
	self.sys.lp['c'][('mwp', 'demand')] = reorder(-Broadcast.seriesToIdx(self.db('mwp'), self.sys.v['demand']), order = self.sys.v['demand'].names)
	self.sys.lp['u'][('loadCap', 'demand')] = self.db('loadHr')
```
We do not specify a lower bound, as the default is simply zero here. The first line specifies that the `c` component is minus the marginal willingness to pay (`mwp[idxCons]`) broadcasted to the full domain of the `demand` variable as declared in `self.sys.v['demand']`. The second line simply declares the upper bound on demand as the auxiliary parameter `loadHr[idxHr, idxCons]` which is already defined over the proper domains. 

**Equality constraints:**

Variational constraints (equality/inequality) are added in a similar manner as the coefficients on parameters:for equality constraints, we iterate through methods following a specific syntax `f'self.initArgsEq_{k}'`where `k` represents the specific constraint (from `self.sys.eq`). For inequality constraints, we simply replace `eq` with `ub`.

In this model, we have a single variational constraint in this model, namely the `equilibrium` one. We note that even though the constraint and variables are now defined over an additional index (`idxHr`) compared to the `MBasic` model, the method that specifies the equilibrium constraint is the same:
```python
def initArgsEq_equilibrium(self):
	self.sys.lazyA('eq2Gen', series = 1,  v = 'generation', constr = 'equilibrium',attr='eq')
	self.sys.lazyA('eq2Dem', series = -1, v = 'demand', constr = 'equilibrium',attr='eq')
```
We rely on the `lazyA` method for adding the coefficients on `generation` and `demand`; this broadcasts the scalars (1 and -1, respectively) to the full domain of the constraint/variable. If the constraint requires a constant different from zero, we would have added it here to the `self.sys.lp['b_eq']` structure. Without adding any, the compiler defaults to zero. 

The `self.compileParams` sets up the structure and returns the `self.sys.out` that can be passed to the linear programming solver: 

In [6]:
m.compileParams();

## 3. Post solution routine - `self.postSolve`

The steps above can be summarized by the following two calls:

In [7]:
m.compile()
sol = m.solve()

The `sol` is the solution object returned from `scipy.optimize.linprog`. Note that the `self.solve` includes an assertion that `sol['status'] == 0`, i.e. it raises an error if the optimization is not successful. For this model, we implement a bigger `postSolve` routine:
```python
def postSolve(self, sol, **kwargs):
	solDict = super().postSolve(sol)
	solDict['surplus'] = -sol['fun']
	solDict['fuelCons'] = fuelConsumption(solDict['generation'], self.db('uFuel'), self.scale)
	solDict['emissions'] = emissionsFuel(solDict['fuelCons'], self.db('uEm'))
	solDict['utilGenCap'] = utilGenCap(solDict['generation'], self.db('genCap'), self.db('idxHr'))
	solDict['utilGenCapHVT'] = utilGenCapHVT(solDict['generation'], self.db('genCapHr'))
	solDict['unitGenCapCosts'] = unitGenC(self.db('mcHr'), self.db('FOM'), self.db('INVC'), solDict['generation'], self.db('genCap'), self.scale)
	solDict['pHr'] = solDict['λeq_equilibrium'] # marginal prices = marginal system costs
	solDict['pAvg'] = avgGenPrice(solDict['generation'], solDict['pHr'], sumOver = ['idxHr','idxGen'])
	solDict['pAvgGen'] = avgGenPrice(solDict['generation'], solDict['pHr'], sumOver = 'idxHr')
	solDict['downlift'] = solDict['pAvg']-solDict['pAvgGen']
	solDict['mEV'] = mEV(solDict['λu_generation'], self.aux_uHrGenCap, self.db('FOM'), self.db('INVC'), self.db('genCap'), self.scale)
	return solDict
```

Beyond the usual solution (variables and duals for variational constrains + domain constraints), this adds a number of additional variables. Note that (almost) all of them are calling local functions defined in the script. We note that another feature of the `MBasicInt` model is that it has a defined `self.scale` which is simply defined as 8760 divided by the number of hours in `idxHr`. The idea is that parameters/variables generally represent hourly values, but variables that measure totals are rescaled as if the model covers a year. This is the case for `fuelCons` and `emissions`, for instance, but it also becomes relevant in the measures for average costs of a generator `unitGenCapCosts` and the marginal economic value `mEV`. These variables weigh the short run costs/revenues relative to long run measured (`FOM`, `INVC`) that are measured on a yearly scale.

The variables are:
* `surplus`: Scalar - simply the negative value of the objective function (which is minimized).
* `fuelCons[idxF]`: Fuel consumption.
* `emissions[idxEm]`: Emissions.
* `utilGenCap[idxGen]`: Utilization rate for generators - measured as $\sum_h E_{h,i}/ H * q_i$ where $H$ is the number of hours in the model.
* `utilGenCapHVT[idxGen]`: Utilitzation rate for generators - measured as $\sum_h E_{h,i} / (\sum_{h}, q_{h,i})$.
* `unitGenCapCosts[idxGen]`: Average costs for a generator taking fixed and investment costs into account. Measured on a yearly scale.
* `pHr[idxHr]`: The dual value of the equilibrium constraint - interpreted as the marginal price in each hour.
* `pAvg`: Average price over the year.
* `pAvgGen[idxGen]`: The average price that different generators receive.
* `downlift[idxGen]`: Difference between average market price and generator-specific price.
* `mEV[idxGen]`: Marginal economic value of a generator, defined as the marginal increase in short run profits by increasing the generating capacity of a plant net of long run costs (`FOM`, `INVC`).

In [8]:
solDict = m.postSolve(sol)
solDict.keys()

dict_keys(['generation', 'demand', 'λeq_equilibrium', 'λl_generation', 'λl_demand', 'λu_generation', 'λu_demand', 'surplus', 'fuelCons', 'emissions', 'utilGenCap', 'utilGenCapHVT', 'unitGenCapCosts', 'pHr', 'pAvg', 'pAvgGen', 'downlift', 'mEV'])

If we want, we can add this directly to our database in `self.db` by calling the `self.postSolveToDB` instead; this draws directly on our customized `self.postSolve` routine and so, we automatically get the new variables with us here:

In [9]:
m.postSolveToDB(sol) # add solution to internal daatbase instead
m.db('generation').head() # look at solution for a bit

idxHr  idxGen       
h01    solar                0.00
       wind_onshore     19794.60
       wind_offshore        0.00
       hydro            37579.86
       nuclear           8640.00
Name: generation, dtype: float64